In [ ]:
#pip install opencv-python
import os
from os import listdir
from os.path import isfile
from os.path import join
from pathlib import Path
import cv2
from math import ceil
import argparse
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
dir = "C:/Users/baner/OneDrive/Desktop/ML2 CF-Net/OCTDL"
#Input 
dataset_folder = dir + "/OCTDL images/"
#lables for each Input
labels_path = dir + "/OCTDL_labels.csv"
#Output
output_folder = dir + "/output"
#Defines the ratio for central cropping of images; 1 means no cropping. default=1
crop_ratio = 1
#Specifies the dimensions to which images should be resized Default is 512
image_dim =512
#validation set
val_ratio = 0.15
#test_ratio
test_ratio = 0.25
#A boolean flag that, when true, indicates images should be padded to become square. Default is False
padding = True
#A boolean flag that, when true, indicates images should be cropped centrally according to the crop_ratio Default is False.
crop = False
#A boolean flag that, when true, indicates images should be resized to image_dim. Default is False.
resize =True

describing the directory and data cleaning parameters for preprocessing of the data

In [ ]:
labels = ['AMD', 'DME', 'ERM', 'NO', 'RAO', 'RVO', 'VID']
folders = ['train', 'val', 'test']

Classes and name of folders for cleaned data

In [ ]:
def padding(img):
    """Returns padded to square image
    """
    height = img.shape[0]
    width = img.shape[1]
    if width == height:
        return img
    elif width > height:
        left = 0
        right = 0
        bottom = ceil((width - height) / 2)
        top = ceil((width - height) / 2)
        result = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0)) #cv2.copyMakeBorder is used to apply this padding, where the new borders are filled with black color
        return result
    else:
        left = ceil((height - width) / 2)
        right = ceil((height - width) / 2)
        bottom = 0
        top = 0
        result = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        return result

takes an image (referred to as img) and adjusts it so that it becomes a square. It ensures that the width and height of the image are the same by adding black-colored borders where necessary.

* It first measures the height and width of the input image. If the width and height are the same, the function just returns the image as it is.

*  If the image has more width than height, it calculates how much padding is needed to make the image square and adds it equally to the top and bottom of the image.

* If the image has more height than width, it calculates the padding needed and adds it equally to the left and right sides of the image.

Note: cv2.copyMakeBorder is used to apply this padding, where the new borders are filled with black color

In [ ]:
def center_crop(img, dim):
    """Returns center cropped image
    """
    width, height = img.shape[1], img.shape[0]
    # process crop width and height for max available dimension
    crop_width = dim[0] if dim[0] < img.shape[1] else img.shape[1]
    crop_height = dim[1] if dim[1] < img.shape[0] else img.shape[0]
    mid_x, mid_y = int(width/2), int(height/2)
    cw2, ch2 = int(crop_width/2), int(crop_height/2)
    crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2]
    return crop_img

This takes an image and a dimensions of the image as input and returns a version of the image that has been cropped around the center to the specified dimensions. 

* It first determines the current width and height of the image by accessing the image’s shape.

*  Then checks the desired crop dimensions provided in dim. It then ensures these dimensions do not exceed the image’s actual size. If the desired dimensions are larger than the image, it keeps the orignal dimensions. 

* It calculates the center point of the image by dividing the width and height by two.

* To make sure the cropping is centered, it calculates half the width and half the height of the crop dimensions. This helps in determining the exact boundaries of the crop area.

* FInally it calculates where to start and stop the crop in both vertical and horizontal directions. It uses the center of the image as a reference point, and crops it

In [ ]:
def scale_image(img, factor=1):
    """Returns resize image by scale factor.
    This helps to retain resolution ratio while resizing.
    Args:
    img: image to be scaled
    factor: scale factor to resize"
    """
    width = int(img.shape[1] * factor)
    height = int(img.shape[0] * factor)
    dsize = (width, height)
    output = cv2.resize(img, dsize, interpolation=cv2.INTER_LANCZOS4)
    return output

The function scale_image is designed to resize an image by a given scale factor, keeping the image's proportions the same. 

Note: cv2.INTER_LANCZOS4 is used for reducing or increasing image size while maintaining high image quality.

In [ ]:
def preprocessing(root_folder, output_folder, file, folder, crop_ratio, dim, label, padding_bool, crop_bool, resize_bool):
    img = cv2.imread(os.path.join(root_folder, label, file))
    if padding_bool:
        img = padding(img)
    if crop_bool:
        img = center_crop(img, (img.shape[1] * crop_ratio, img.shape[0] * crop_ratio))
    if resize_bool:
        img = cv2.resize(img, dim, interpolation=cv2.INTER_LANCZOS4)    #cv2.INTER_LANCZOS4 is used for reducing or increasing image size while maintaining high image quality.
    cv2.imwrite(os.path.join(output_folder, folder, label, Path(file).name), img, [cv2.IMWRITE_JPEG_QUALITY, 100])  #image is saved with high JPEG quality (100), which means minimal compression and maximal quality.

Calls the functions for resizing, cropping and padding, based on the padding_bool, crop_bool and resize_bool. Then saves the images in output folder -> file_name( train, test, and val) -> disease class -> image name

In [ ]:
root_folder = dataset_folder
train_ratio = 1 - val_ratio - test_ratio
dim = (image_dim, image_dim)
padding_bool = padding
crop_bool = crop
resize_bool = resize
#Reading the Labels
df = pd.read_csv(labels_path)

In [ ]:
for folder in folders:
    for label in labels:
        Path(os.path.join(output_folder, folder, label)).mkdir(parents=True, exist_ok=True)

Creates the output folder

In [ ]:
for label in tqdm(labels):
        df_label = df[df['disease'] == label][['file_name', 'disease', 'patient_id']]
        patients_list = df_label.patient_id.unique()
        if len(patients_list) < 10:
            print(f"Skipping {label} due to insufficient data.")
            continue
        train_patients, test_patients = train_test_split(patients_list, test_size=1 - train_ratio)
        val_patients, test_patients = train_test_split(test_patients, test_size=test_ratio / (test_ratio + val_ratio))
        df_label_train = df_label[df_label['patient_id'].isin(train_patients)]
        df_label_val = df_label[df_label['patient_id'].isin(val_patients)]
        df_label_test = df_label[df_label['patient_id'].isin(test_patients)]
        
        print(label, len(df_label_train), len(df_label_val), len(df_label_test))

Divides the dataset into training, validation, and test subsets based on patient data for different disease classes.

* It loops through each unique disease class stored in labels. The progress of this loop is tracked using tqdm, (displays a progress bar).
* For each disease class, it selects image names, classes, and patient IDs from df that match the current label.
* The code identifies unique patient IDs associated with each label. If a disease class has fewer than 10 unique patients
* It splits the list of patient IDs into training validation, and test groups based on a ratio 
* Creates 3 new df for each train, test and val dataset

In [ ]:
for i in range(0, len(df_label_train)):
    file_name = df_label_train.iloc[i, 0] + '.jpg'
    file_label = df_label_train.iloc[i, 1]    
    preprocessing(root_folder, output_folder, file_name, 'train', crop_ratio, dim, file_label, padding_bool, crop_bool, resize_bool)

for i in range(0, len(df_label_test)):
    file_name = df_label_test.iloc[i, 0] + '.jpg'
    file_label = df_label_test.iloc[i, 1]    
    preprocessing(root_folder, output_folder, file_name, 'test', crop_ratio, dim, file_label, padding_bool, crop_bool, resize_bool)    

for i in range(0, len(df_label_val)):
    file_name = df_label_val.iloc[i, 0] + '.jpg'
    file_label = df_label_val.iloc[i, 1]    
    preprocessing(root_folder, output_folder, file_name, 'val', crop_ratio, dim, file_label, padding_bool, crop_bool, resize_bool) 

Executes the function preprocessing for each df(train, validation, and test) which in-turn calls the functions for resizing, cropping and padding.